# Notebook から Snowpark を使ってみよう

## Snowpark セッション作成

ますはセッションを作成しましょう。

`Session` は、Snowsight などの Web UI や Snowflake クライアントで接続するときに作成されるセッションの概念を模したオブジェクトです。

※今回のハンズオンでは、接続情報を直接記載して Snowflake にアクセスしますが、実際に運用するプロダクトなどでは、基本的に直節記載は避けたほうが良いでしょう。AWS SecretManager などの機密情報管理サービスや、Github secret を使って、不用意に開示しないよう管理しましょう。

In [ ]:
from snowflake.snowpark.session import Session
from config import connection_parameters

# config.py の内容：
# connection_parameters = {
#     'account': '<org_name>-<account_name>',  # お使いの Snowflake アカウントの識別子
#     'user': '<your_username>',  # Snowflake アカウントにサインインするユーザー名
#     'password': '<your_password>',  # ユーザーのパスワード
#     'role': 'SYSADMIN',  # ハンズオンで使用するロール。変更は不要です
#     'database': 'DEGEEKS_HO_DB',  # ハンズオンで使用するデータベース。存在しなければ新規作成します（「gegeeks_ho_notebook.ipynb」をご参照ください）
#     'schema': 'PUBLIC',  # ハンズオンで使用するスキーマ
#     'warehouse': 'DEGEEKS_HO_WH'  # ハンズオンで使用するウェアハウス。存在しなければ新規作成します（「gegeeks_ho_notebook.ipynb」をご参照ください）
# }

session = Session.builder.configs(connection_parameters).create()
print(session)

今回のハンズオンで使用するデータベースを作成します。


In [ ]:
session.sql(f"create database if not exists {connection_parameters['database']}").collect()
session.sql(f"use database {connection_parameters['database']}").collect()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

今回のハンズオンで使用するウェアハウスを作成します。

In [ ]:
session.sql(f"""
    create warehouse if not exists {connection_parameters['warehouse']} with
        warehouse_size = xsmall
        auto_suspend = 60
        auto_resume = true
""").collect()
session.sql(f"use warehouse {connection_parameters['warehouse']}").collect()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

In [ ]:
session.sql(f"use database {connection_parameters['database']}").collect()
session.sql(f"use schema {connection_parameters['schema']}").collect()
session.sql(f"use warehouse {connection_parameters['warehouse']}").collect()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

## TPC-H をクエリしてみよう

早速、Snowpark Dataframe を作成してみましょう。

In [ ]:
import pandas as pd

df = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS")
df.limit(5).to_pandas()
# pd.DataFrame(df.limit(5).collect()) # 上のコードでエラーになる場合

続けて、Dataframe を操作してみましょう。


フィルタ（Where句）はこのように記述します。

In [ ]:
import snowflake.snowpark.functions as F


df.where(
    F.col('O_TOTALPRICE') > 500000
).select(
    F.col('O_ORDERKEY'), F.col('O_TOTALPRICE'), F.col('O_ORDERDATE')
).show(50)

ソートはこのように記述します。

In [ ]:
df.select([
    F.col('O_ORDERKEY'), F.col('O_TOTALPRICE'), F.col('O_ORDERDATE')]
).sort(
    F.col('O_TOTALPRICE').desc()
).show(10)

グルーピングと集計関数は、このように記述します。

In [ ]:
df.group_by(
    F.col('O_ORDERPRIORITY')
).agg(
    F.count(F.col('O_ORDERKEY')).as_('ORDER_CNT'),
    F.max(F.col('O_TOTALPRICE')).as_('TOTALPRICE_MAX'),
    F.min(F.col('O_TOTALPRICE')).as_('TOTALPRICE_MIN'),
    F.sum(F.col('O_TOTALPRICE')).as_('TOTALPRICE_SUM'),
    F.avg(F.col('O_TOTALPRICE')).as_('TOTALPRICE_AVG')
).show()

基本統計量を算出することもできます。

In [ ]:
# df.describe().show()  # ちょっと見づらいので pandas 化する
df.describe().to_pandas()

ここで、`to_pandas()` が登場したので、少々横道にそれますが、Pandas Dataframe と Snowpark Dataframe の違いを見ておきましょう。
Python 上で使用しているメモリサイズを出力すると、、、

In [ ]:
import sys
import numpy as np


pdf = pd.DataFrame(df.collect())
print(f'Size of Pandas DataFrame in Memory: {np.round(sys.getsizeof(pdf) / (1024.0**2), 2)} MB ({sys.getsizeof(pdf)} B)')
print(f'Size of Snowpark DataFrame in Memory: {np.round(sys.getsizeof(df) / (1024.0**2), 2)} MB ({sys.getsizeof(df)} B)')

Pandas Dataframe はデータの実体をメモリ上に保持していますが、Snowpark Dataframe は保持していないようです。

[Snowpark のドキュメント](https://docs.snowflake.com/ja/developer-guide/snowpark/python/working-with-dataframes)で説明されている通り、Snowpark Dataframe はデータを取得するために必要な処理をまとめた、いわばクエリの塊のようなものです。特定のアクション（`collect()` など）が実行されるまで、処理は行われません。アクションが実行されたときに、変換処理が実行され、結果を取得します。これが遅延評価の特徴です。

## UDFとストアドプロシージャ


次に、UDFとストアドプロシージャを作ってみます。  
Notebook 上から、Python で記述した関数を UDF やストアドプロシージャとして Snowflake に登録することができます。  

### UDF 

UDF（User Defined Function）とは、データベースシステムに組み込まれているシステム関数（`count()`、 `sqrt()` 等）に対し、ユーザー自身が定義する関数のことです。  
ここでは、文字の全角・半角を正規化する関数を作ってみましょう。

In [ ]:
from snowflake.snowpark.functions import udf
import unicodedata


@udf(
    name='unicode_nfkc_udf',
    is_permanent=True,
    stage_location='@~/sample_udf_stage',
    replace=True
)
def unicode_nfkc_udf(x: str) -> str:
    return unicodedata.normalize('NFKC', x)


In [ ]:
# show() では表形式になる
session.sql(
    """
    select
        '全角半角ｶﾅ混じり　表記ゆれてるＹＯ!' as input,
        unicode_nfkc_udf(input) as output"""
).show()

### ストアドプロシージャ
次はストアドプロシージャを作ってみましょう。ストアドプロシージャも UDF と同様、コードを書くことで SQL を拡張できるものです。 管理操作を実行させたい、などの場合はこちらを使います。

TPC-H の ORDERS テーブルを集計するストアドプロシージャを作ってみましょう。


In [ ]:
from snowflake.snowpark.functions import sproc


@sproc(
    name="count_sproc",
    packages=['snowflake-snowpark-python'],
    is_permanent=True,
    stage_location="@~/sample_sproc_stage",
    replace=True
)
def count_sproc(session: Session, column: str) -> int:
    df = session.table('SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS')
    return df.agg(F.count(F.col(column))).collect()[0][0]

In [ ]:
print(count_sproc(session, 'O_ORDERKEY'))

---


## Appendix

### ハンズオン資料に載せていないクエリ集